In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#load daily changes df
daily_changes = pd.read_csv('../data/processed/daily_changes.csv')
daily_changes = daily_changes.drop('Unnamed: 0', axis=1)
daily_changes.tail()


,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,...,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d,block_height,halving_period,months,years
3852,-15164.0,-3.0,-4290100.0,-2567.640654,0.149137,3.693922e+10,1.116876e+09,0.000000e+00,-0.000008,0.501252,...,2.146052,3169.275239,1.130253e+09,0.000148,0.000032,0.000326,150.0,0.0,0.0,0.0
3853,39665.0,10.0,11091805.0,-11868.947500,-0.055382,-8.325704e+09,1.367981e+09,0.000000e+00,0.000057,1.944686,...,10.137357,150149.817308,5.344392e+09,0.000024,-0.001156,0.000097,160.0,0.0,0.0,0.0
3854,30792.0,9.0,15013126.0,20285.169157,0.048055,1.709948e+10,2.001681e+09,0.000000e+00,-0.000048,-1.406494,...,-25.373569,-38165.333594,-8.441732e+08,0.000014,-0.001518,-0.000042,169.0,0.0,0.0,0.0
3855,-110220.0,-10.0,-16400302.0,-20913.856500,0.090001,2.417704e+10,1.121651e+09,5.801295e+11,-0.000057,-1.716026,...,-2.779499,-186265.643460,-6.517472e+09,0.000050,-0.001126,-0.000509,159.0,0.0,0.0,0.0
3856,12262.0,-3.0,13150902.0,109042.692791,-0.043088,-6.269335e+09,1.159979e+09,3.073534e+10,-0.000051,-2.098584,...,-27.911835,-17407.250465,-8.276311e+08,-0.000163,-0.000504,0.000083,156.0,0.0,0.0,0.0


In [3]:
! pip install --user tscv

In [5]:
#import package to perform train-test split
from tscv import GapLeavePOut, GapKFold, GapWalkForward, gap_train_test_split

In [23]:
#train-test split using varios methods from the tscv package

#Gap leave p out
p= round(len(daily_changes) * 0.2)
gap_before = round(p * 0.2)
gap_after = p - gap_before

cv_pout = GapLeavePOut(p=p, gap_before=gap_before, gap_after=gap_after)
pout_split = cv_pout.split(daily_changes)
pout_split




<generator object GapCrossValidator.split at 0x7ff77e103190>

In [7]:
#Gap k fold
cv_gapkfold = GapKFold(n_splits=5, gap_before=gap_before, gap_after=gap_after)
gapkfold_split = cv_gapkfold.split(daily_changes)
gapkfold_split

<generator object GapCrossValidator.split at 0x7ff77e17ae40>

In [24]:
#Gap kwalk forward
test_size= len(daily_changes) * 0.2
gap_size = test_size * 0.1


cv_gapwf = GapWalkForward(n_splits=5, gap_size=gap_size, test_size=test_size)
gapfw_split = cv_gapwf.split(daily_changes)
gapfw_split

<generator object GapWalkForward.split at 0x7ff77c28bac0>

In [25]:
#Gap train-test split

X = daily_changes.drop('PriceUSD', axis=1)
y = daily_changes['PriceUSD']

X_train, X_test, y_train, y_test = gap_train_test_split(X, y, test_size=test_size, gap_size=gap_size)

/home/julio/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [26]:
y_test

2530      47.386200
2531      -0.777043
2532    -123.681322
2533     -64.460207
2534     -71.363890
           ...     
3852    1982.224297
3853    -449.170269
3854     916.210671
3855    1296.313473
3856    -338.714899
Name: PriceUSD, Length: 1327, dtype: float64

In [27]:
X_train

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,...,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d,block_height,halving_period,months,years
0,69.0,2.0,2365.0,4.444066,-83.473202,-1.667395e+04,2.437326e+03,0.000000e+00,-0.003036,-0.000261,...,0.053296,2.520063e+04,1.938846e+03,0.000000,0.000000,0.000000,174.0,0.0,0.0,0.0
1,7.0,7.0,58171.0,290.435512,-17.458500,-2.028480e+04,1.278047e+03,0.000000e+00,-0.000150,-0.000012,...,-0.109338,-1.313271e+04,-1.251458e+03,0.000000,0.000000,0.000000,181.0,0.0,0.0,0.0
2,-152.0,26.0,-72434.0,-486.924600,-6.749070,1.626607e+04,1.434370e+03,1.023182e-12,0.000000,0.000000,...,1.362306,-4.309260e+03,-2.013698e+02,0.000000,0.000000,0.000000,207.0,0.0,0.0,0.0
3,-190.0,-31.0,-24241.0,-31.378074,-13.225212,-7.151690e+04,9.307358e+02,-1.023182e-12,0.000000,0.000000,...,-1.406014,1.169681e+04,1.227898e+02,0.000000,0.000000,0.000000,176.0,0.0,0.0,0.0
4,61.0,18.0,2251.0,-41.510192,-1.941361,7.986150e+03,1.023022e+03,0.000000e+00,0.000045,0.000003,...,0.243910,-1.761585e+04,-9.850738e+02,0.000000,0.000000,0.000000,194.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,-89140.0,24.0,-18407030.0,-222511.560386,-0.003216,-5.527485e+05,9.219286e+06,0.000000e+00,-0.000008,-0.003451,...,4.674088,-9.719327e+05,-4.436621e+08,-0.000014,-0.000071,0.000007,162.0,0.0,0.0,0.0
1986,48342.0,9.0,9391220.0,27946.684211,-0.122804,-5.608472e+08,-4.271729e+06,0.000000e+00,0.000010,-0.001880,...,-31.923680,-1.077408e+06,-5.863230e+08,0.000622,0.001937,0.001618,171.0,0.0,0.0,0.0
1987,-7585.0,34.0,-260095.0,-90900.812124,0.014776,6.947200e+07,1.969846e+06,0.000000e+00,-0.000006,-0.001818,...,-1.730073,-1.402894e+06,-5.816111e+08,-0.000048,0.000009,-0.000109,205.0,0.0,0.0,0.0
1988,5075.0,-54.0,8157214.0,214779.556905,-0.005403,-2.322903e+07,8.365064e+05,0.000000e+00,0.000011,0.004424,...,7.438654,5.820914e+05,2.432079e+08,-0.000050,-0.000041,-0.000146,151.0,0.0,0.0,0.0


In [28]:
X_test

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,...,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d,block_height,halving_period,months,years
2530,-26357.0,-1.0,337025.0,9497.106475,0.032221,7.819877e+08,1.049476e+08,0.000000e+00,-0.000002,0.072313,...,2.404616,-384224.356629,-9.187050e+08,-0.000064,-0.001724,-0.000006,139.0,0.0,0.0,0.0
2531,-10857.0,21.0,13851758.0,-43701.312725,-0.015844,-7.376168e+06,7.267168e+07,0.000000e+00,-0.000340,-0.913648,...,-1.858953,-149810.461986,-4.039353e+08,0.000003,-0.001249,0.000023,160.0,0.0,0.0,0.0
2532,-79804.0,-14.0,-13995653.0,-4873.237586,-0.142480,-2.024645e+09,1.369832e+07,0.000000e+00,0.000028,-0.091687,...,-12.969746,-412601.504764,-1.287540e+09,0.000190,0.000207,0.000649,146.0,0.0,0.0,0.0
2533,-100525.0,-24.0,-23789323.0,-9290.446328,-0.075211,-1.053950e+09,1.302836e+07,9.765625e-04,0.000108,0.182616,...,2.489398,181728.024484,3.604361e+08,0.000040,0.000126,0.000249,122.0,0.0,0.0,0.0
2534,167726.0,20.0,26146859.0,52484.231009,-0.082125,-1.166847e+09,9.203870e+06,-9.765625e-04,-0.000016,-0.141230,...,17.019859,680665.723785,1.535093e+09,-0.000099,-0.003885,0.000151,142.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852,-15164.0,-3.0,-4290100.0,-2567.640654,0.149137,3.693922e+10,1.116876e+09,0.000000e+00,-0.000008,0.501252,...,2.146052,3169.275239,1.130253e+09,0.000148,0.000032,0.000326,150.0,0.0,0.0,0.0
3853,39665.0,10.0,11091805.0,-11868.947500,-0.055382,-8.325704e+09,1.367981e+09,0.000000e+00,0.000057,1.944686,...,10.137357,150149.817308,5.344392e+09,0.000024,-0.001156,0.000097,160.0,0.0,0.0,0.0
3854,30792.0,9.0,15013126.0,20285.169157,0.048055,1.709948e+10,2.001681e+09,0.000000e+00,-0.000048,-1.406494,...,-25.373569,-38165.333594,-8.441732e+08,0.000014,-0.001518,-0.000042,169.0,0.0,0.0,0.0
3855,-110220.0,-10.0,-16400302.0,-20913.856500,0.090001,2.417704e+10,1.121651e+09,5.801295e+11,-0.000057,-1.716026,...,-2.779499,-186265.643460,-6.517472e+09,0.000050,-0.001126,-0.000509,159.0,0.0,0.0,0.0


In [29]:
y_train

0       -0.005040
1       -0.006064
2        0.004457
3       -0.020723
4        0.002123
          ...    
1985    -0.159887
1986   -37.497294
1987     4.484363
1988    -1.653015
1989    10.287142
Name: PriceUSD, Length: 1990, dtype: float64